Once the model is packaged, it needs to be deployed using an appropriate infrastructure.To ensure smooth deployment across different environments, the model is often packaged using:
- Virtual Environments: Use venv or conda to manage dependencies in different environments. 
- Model APIs: Expose the model via REST APIs using Flask, FastAPI, Django REST framework.
- Containerization:
    1) Docker: Enables portability, reproducibility and isolated environments. EPackages model, dependencies, and API into a lightweight container.
    2) Kubernetes: For scaling deployments in cloud environments.
- Selecting the appropriate deployment pattern, such as:
    1) Batch Processing: Best for periodic predictions (e.g., fraud detection, risk analysis).
    2) Real-Time Inference (APIs): For applications needing instant responses (e.g., recommendation systems, chatbots).
    3) Edge Deployment : Running models on IoT or mobile devices (e.g., image recognition on phones).
- Implementing Infrastructure as Code (IaC) (e.g., Terraform, AWS CloudFormation) for reproducible deployments.
- Cloud-Specific Deployment:
    1) AWS: Deploy via SageMaker, Lambda, ECS/Fargate, S3 (batch), API Gateway.
    2) Azure: Use Azure ML, Azure Functions, AKS (Kubernetes), Azure Batch.
    3) GCP: Deploy with Vertex AI, Cloud Functions, AI Platform, Cloud Run.
    4) On-Premise / Hybrid: Use Kubernetes (K8s), OpenShift, MLflow for flexible deployments.



In [ ]:
# Deploying a model as an API with FastAPI: When you need real-time inference.Also good for web aplications
##Create a Flask API for the model 

from flask import Flask, request, jsonify
import joblib
import numpy as np

app = Flask(__name__)

# Load the trained model
model = joblib.load('random_forest.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    prediction = model.predict(np.array(data["features"]).reshape(1, -1))
    return jsonify({'prediction': int(prediction[0])})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

